In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
# 2. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# STEP 1:


Let a reference image is X and an image to be processed is Z.

Convert X : [R, G, B] to [Y, Cb, Cr]

Convert Z : [R, G, B] to [Y, Cb, Cr]

Apply histogram specification on h[ Zy ] to match with h[ Xy ]. Accordingly
Z is changed to Z'.

Convert Z'  : [Y, Cb, Cr] to [R, G, B]

### BELOW, WE ARE IMPORTING ALL REQUIRED LIBRARIES

In [3]:
import numpy as np
from PIL import Image,ImageEnhance
import cv2
import sys
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import skimage.viewer
from skimage import data,io,color
from skimage import exposure
from skimage.exposure import match_histograms
from skimage import img_as_ubyte
from scipy import stats
from matplotlib.pyplot import figure

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Viewer requires Qt
  import sys


In [ ]:
def enhancer(im,a):
    #im = Image.open("drive/My Drive/Results 3/result0244 enhanced-a=0.6,b=0.5,g=-0.05,d=0.05 .JPG")
    enhancer = ImageEnhance.Brightness(im)
    enhanced_im = enhancer.enhance(a)
    enhanced_im.save("drive/My Drive/dummy/dummyEnhance.JPG")
    return 

### THE BELOW CODE IS A FUNCTION IN ORDER TO FILTER OUT THE SKY AND SUPERIMPOSE IT ON THE FINAL IMAGE TO KEEP THE SKY UNCHANGED

In [ ]:
def skyfilter(orig,new):
    l = orig.shape[0]
    b = orig.shape[1]
    sumb = np.sum(orig[:,:,2])
    x = (sumb/(l*b))
    for i in range(l):
        print(i)
        for j in range(b):
            if orig[i][j][2] > (1*x):  
                new[i][j] = orig[i][j]
    
    return new

### THE BELOW CODE IS TO PERFORM HISTOGRAM MATCHING OF LUMINANCE ALONE

In [ ]:
ref1 = io.imread("drive/My Drive/Images_for_Adarsh/DSC_6139.JPG")
orig1 = io.imread("drive/My Drive/Images_for_Adarsh/DSC_6138.JPG")
ref = color.rgb2ycbcr(ref1)
orig = color.rgb2ycbcr(orig1)
f1 = match_histograms(orig, ref, multichannel=True)
f1[:,:,1] = orig[:,:,1]
f1[:,:,2] = orig[:,:,2]
f2 = color.ycbcr2rgb(f1)
io.imsave("drive/My Drive/dummy/dummy.JPG", f2)
f3 = io.imread("drive/My Drive/dummy/dummy.JPG")
final = skyfilter(orig1,f3)

# io.imshow(matched)
io.imsave("drive/My Drive/Y Histogram_Match RESULTS/Y6138.JPG", final)
#viewer = skimage.viewer.ImageViewer(ref)
#viewer.show()


# STEP 2: 
Again the reference image is X, but the image to be processed is Z'.

Convert X : [R, G, B] to [H, S, V]

Convert Z' : [R, G, B] to [H, S, V]


### THIS CODE CREATES A MASK FOR THE TEMPLE REGION ALONE INORDER TO AVOID SKY PART

In [4]:
def createmask(Img,v):                               #the image is accepted in BGR format
    l = Img.shape[0]
    b = Img.shape[1]
    sumb = np.sum(Img[:,:,0])
    x = (sumb/(l*b))
    new = 255 * np.ones((l,b),np.uint8)
    for i in range(l):
        for j in range(b):
            if Img[i][j][0] > (v*x):  
                new[i][j] = 0
    #cv2_imshow(new)
    return new

In [5]:
ref = cv2.imread("drive/My Drive/Images_for_Adarsh/DSC_6139.JPG")
imgp = cv2.imread("drive/My Drive/LATEST RESULTS/Y Histogram_Match RESULTS_LATEST/Y0690.JPG")

mask1 = createmask(ref,1.75)
mask2p = createmask(imgp,1.5)

In [ ]:
cv2_imshow(mask2p)

### THE CODE BELOW PERFORMS THE MATCHING OF MODES BY MULTIPLYING THE UE VALUES OF THE IMAGE TO BE PROCESSED BY A FACTOR:
### FACTOR = mode of ref img/mode of img to be processed

In [ ]:
refhsv = cv2.cvtColor(ref,cv2.COLOR_BGR2HSV)
imgphsv = cv2.cvtColor(imgp,cv2.COLOR_BGR2HSV)
href = cv2.calcHist([refhsv],[0],mask1,[256],[0,256])
himgp = cv2.calcHist([imgphsv],[0],mask2p,[256],[0,256])
# f = figure(num=None, figsize=(9, 5), dpi=400, facecolor='w', edgecolor='k')
# plt.subplot(1,2,1)
# plt.plot(href, color = 'r')
# plt.subplot(1,2,2)
# plt.plot(himgp, color = 'b')
mref = np.argmax(href)
mimgp = np.argmax(himgp)
factor = mref/mimgp
imgphsv_step1 = imgphsv
imgphsv_step1[:,:,0] = factor * imgphsv[:,:,0]

vref = np.mean(np.abs(refhsv[:,:,0] - mref))
